# Required Libraries
In the code chunk below, we introduce all the Python libraries that were used for our analysis. Note that Python Version 3.8.12 is used throughout this analysis.

In [21]:
import pandas as pd

from pycaret.classification import *
import imblearn as im
import sklearn

from pandas_profiling import ProfileReport

# Reading and Exploring the Dataset

## Reading the Data

In [13]:
df = pd.read_csv('data/features_10_sw.csv')
df.columns = df.columns.str.lower()
df.head()

,fatigue status,subject id,activity,position,order of time,min_x,min_y,min_z,min_abs,max_x,max_y,max_z,max_abs,average_x,average_y,average_z,average_abs,median_x,median_y,median_z,median_abs,std_x,std_y,std_z,std_abs,var_x,var_y,var_z,var_abs,range_x,range_y,range_z,range_abs
0,pref,1,walk,ankle,1,-1.666586,-3.369942,-1.004515,0.543995,3.529510,0.133768,0.899586,4.285603,-0.219868,-1.333620,-0.036678,1.604442,-0.272879,-1.129949,-0.012175,1.253170,0.846181,0.739588,0.357869,0.802703,0.716021,0.546990,0.128070,0.644331,5.196096,3.503710,1.904101,3.741608
1,pref,1,walk,ankle,1,-1.719703,-3.467588,-1.201145,0.639574,3.741023,0.042868,0.959449,4.551759,-0.217683,-1.412366,-0.041082,1.708210,-0.293303,-1.200664,-0.007178,1.323875,0.941959,0.774067,0.410574,0.883193,0.887288,0.599179,0.168571,0.780031,5.460726,3.510456,2.160594,3.912185
2,pref,1,walk,ankle,1,-1.787620,-3.527154,-1.160556,0.648391,3.729927,0.060303,0.913621,4.548308,-0.199166,-1.430649,-0.027728,1.726646,-0.279804,-1.211285,-0.010637,1.337642,0.967384,0.784440,0.395116,0.901268,0.935832,0.615346,0.156117,0.812283,5.517546,3.587457,2.074177,3.899917
3,pref,1,walk,ankle,1,-1.795576,-3.502021,-1.111818,0.645545,3.954491,0.111993,0.920629,4.724261,-0.198807,-1.427603,-0.037922,1.724429,-0.271068,-1.207968,-0.017982,1.328807,0.969760,0.783312,0.386223,0.898690,0.940434,0.613578,0.149168,0.807644,5.750066,3.614015,2.032447,4.078716
4,pref,1,walk,ankle,1,-1.756625,-3.475091,-1.153110,0.633805,3.684966,0.023153,0.933295,4.511043,-0.204406,-1.440369,-0.038216,1.736274,-0.287479,-1.222287,-0.012045,1.345195,0.958411,0.793327,0.413299,0.906142,0.918551,0.629367,0.170816,0.821093,5.441591,3.498245,2.086405,3.877238


## ETL

In [20]:
# Deleting subject 21 from the data
df = df[df['subject id'] != 21]

## EDA


### Summary Statistics

In [24]:
df.describe()

,subject id,order of time,min_x,min_y,min_z,min_abs,max_x,max_y,max_z,max_abs,average_x,average_y,average_z,average_abs,median_x,median_y,median_z,median_abs,std_x,std_y,std_z,std_abs,var_x,var_y,var_z,var_abs,range_x,range_y,range_z,range_abs
count,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03,3.456000e+03
mean,11.312500,2.000000,-1.401003,-0.878119,-0.460208,0.438214,1.221823,1.068818,0.716911,2.750898,-0.133027,0.091572,0.149354,1.243133,-0.137607,0.080065,0.148898,1.130510,5.062614e-01,4.177705e-01,2.054276e-01,5.277572e-01,3.746927e-01,2.246334e-01,5.627159e-02,3.570939e-01,2.622827e+00,1.946937e+00,1.177118e+00,2.312684e+00
std,6.550936,0.816615,1.226541,1.218504,0.494429,0.165365,1.468955,1.238918,0.371278,1.394188,0.588939,0.959869,0.117020,0.287675,0.608604,0.866212,0.105919,0.117371,3.441313e-01,2.238655e-01,1.186387e-01,2.803373e-01,4.272243e-01,2.198713e-01,7.914808e-02,3.410199e-01,1.989586e+00,9.927003e-01,7.616659e-01,1.343370e+00
min,1.000000,1.000000,-4.827109,-4.028788,-2.703763,0.010336,-0.974377,-1.004506,0.018535,1.007776,-1.020154,-1.474669,-0.436342,0.980503,-1.161834,-1.281279,-0.531413,0.928176,1.430000e-17,1.400000e-17,5.330000e-16,4.970000e-16,2.060000e-34,1.960000e-34,2.840000e-31,2.470000e-31,9.890000e-17,9.890000e-17,1.330000e-15,1.330000e-15
25%,5.500000,1.000000,-1.778043,-1.589218,-0.711075,0.341115,0.057949,0.030646,0.493646,1.729405,-0.759342,-0.977855,0.090635,1.035966,-0.759305,-0.965053,0.097357,1.041402,2.283322e-01,2.525500e-01,1.262035e-01,3.292679e-01,5.213559e-02,6.378152e-02,1.592733e-02,1.084174e-01,1.159902e+00,1.225594e+00,6.396746e-01,1.335520e+00
50%,12.000000,2.000000,-1.446452,-0.521516,-0.295230,0.412624,0.848179,0.689463,0.631701,1.890966,-0.059058,0.071288,0.164498,1.059310,-0.104239,0.058237,0.153379,1.090072,3.471700e-01,3.206885e-01,1.713916e-01,3.829455e-01,1.205270e-01,1.028411e-01,2.937507e-02,1.466473e-01,1.443941e+00,1.468542e+00,9.368592e-01,1.566250e+00
75%,18.000000,3.000000,-0.413176,0.065785,-0.113111,0.508120,1.768755,1.741795,0.833902,4.462704,0.240485,0.964982,0.216872,1.602260,0.253038,0.969918,0.201203,1.226128,9.360912e-01,6.492838e-01,2.437756e-01,8.651854e-01,8.762668e-01,4.215694e-01,5.942657e-02,7.485458e-01,5.173271e+00,2.912041e+00,1.451232e+00,3.886833e+00
max,20.000000,3.000000,0.701802,0.888078,1.021000,1.021079,5.063914,4.053110,2.502131,5.887368,1.025848,1.456620,1.021812,1.830450,1.079681,1.282438,1.023488,1.593939,1.181909e+00,8.958869e-01,7.924012e-01,1.116514e+00,1.396908e+00,8.026133e-01,6.278997e-01,1.246603e+00,6.666339e+00,4.328798e+00,4.506593e+00,5.260688e+00


### Quick Visuals
Capitalizing on the [pandas profiling library](https://pandas-profiling.ydata.ai/docs/master/index.html), we have generated a detailed EDA report of our dataset (after subject 21 was dropped). The exported HTML report is titled `eda_report` and it is saved in the `results` subfolder.

In [25]:
design_report = ProfileReport(df)
design_report.to_file(output_file='results/eda_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Machine Learning Experiment for the Ankle Data

## Setup

In [33]:
ankle_df = df[df.position == 'ankle']

exp_10sec_ankle = setup(data = ankle_df, 
                       target = 'fatigue status',
                       train_size = 0.8,
                       categorical_features = ['order of time'],
                       ignore_features = ['activity', 'position', 'subject id', 'order of time'],
                       transformation = True, transformation_method = 'yeo-johnson',
                       remove_multicollinearity = True, multicollinearity_threshold = 0.9,
                       feature_selection = True, feature_selection_method = 'classic',
                       fold_strategy = 'groupkfold', fold = 5, fold_groups = 'subject id',
                       session_id = 2022,
                       log_experiment = True)

,Description,Value
0,session_id,2022
1,Target,fatigue status
2,Target Type,Binary
3,Label Encoded,"post: 0, pref: 1"
4,Original Data,"(1188, 33)"
5,Missing Values,False
6,Numeric Features,28
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


## Training Multiple Classifiers

In [39]:
best_models = compare_models(include = ['lr', 'knn', 'nb', 'dt', 'ridge', 'lda', 'qda'], n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9438,0.9836,0.9631,0.9267,0.9440,0.8874,0.8889,0.0140
dt,Decision Tree Classifier,0.9083,0.9076,0.9039,0.9102,0.9070,0.8153,0.8155,0.0080
qda,Quadratic Discriminant Analysis,0.8209,0.9153,0.9291,0.7631,0.8376,0.6405,0.6564,0.0060
lr,Logistic Regression,0.7735,0.8387,0.8045,0.7629,0.7805,0.5479,0.5527,0.0100
lda,Linear Discriminant Analysis,0.7731,0.8518,0.8652,0.7341,0.7928,0.5448,0.5557,0.0080
ridge,Ridge Classifier,0.7579,0.0000,0.8334,0.7292,0.7758,0.5151,0.5235,0.0040
nb,Naive Bayes,0.6844,0.7475,0.7802,0.6582,0.7127,0.3630,0.3712,0.0060


## Analyze Each of the Top 3 Models

### kNN

In [40]:
evaluate_model(best_models[0])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

### DT

In [41]:
evaluate_model(best_models[1])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

### QDA

In [42]:
evaluate_model(best_models[2])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Holdout Performance

### kNN

In [43]:
predict_model(best_models[0])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9412,0.9919,0.9492,0.9333,0.9412,0.8824,0.8825


,average_x,var_abs,range_z,max_x,min_z,max_z,median_abs,range_y,std_x,median_x,average_abs,min_abs,median_z,range_x,max_y,max_abs,fatigue status,Label,Score
0,0.302809,0.383175,2.329275,-1.125984,-2.593261,1.549342,-0.115828,1.328049,-1.737958,0.745661,-0.254219,-1.382665,2.201327,-1.816205,1.101744,0.133481,post,post,1.0
1,0.820430,0.231805,-0.433777,-0.847015,0.267348,-0.440247,-0.525523,-0.122850,0.822879,1.286442,-0.228252,0.197210,0.287812,0.451247,0.936357,0.110321,pref,pref,1.0
2,-0.097086,-0.362411,0.349375,-1.198545,0.142291,0.768653,-0.241917,-0.580970,-0.227414,0.630337,-0.682743,-0.045984,0.759261,-0.530719,0.873072,-0.442057,post,post,1.0
3,0.882774,-0.248497,-1.332795,-0.906435,1.192027,-0.890347,-0.798565,0.446981,-0.730281,1.055298,-0.682070,0.036868,0.416936,-0.566333,1.002208,-0.490266,pref,pref,1.0
4,0.251515,-2.499033,-2.204331,-1.242056,1.455820,-2.134167,-2.269894,-2.277633,-2.281506,-0.019361,-2.388288,1.492297,-0.741366,-2.609826,-1.567937,-2.647125,post,post,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,1.452866,0.929014,-1.093194,-0.848462,0.140169,-2.135501,-0.773128,1.211372,-0.201790,1.416315,-0.071178,0.108084,-2.251777,0.086417,1.131024,0.615118,pref,pref,1.0
234,0.647454,1.213704,1.609738,0.718750,-1.915458,0.893496,0.739140,2.149094,-0.560048,0.017677,0.847435,-0.688200,1.386721,1.109594,-0.656734,-0.236694,post,post,1.0
235,0.968396,0.153740,-1.059784,-0.849844,0.788706,-0.931693,-0.951620,-0.020578,0.128617,1.212644,-0.539768,0.358486,0.380109,0.101774,0.960143,-0.136314,pref,pref,1.0
236,0.819841,0.008660,-1.383519,-0.845820,1.298063,-0.843920,-0.694902,0.238703,-0.214410,1.102260,-0.349707,0.013334,0.401627,-0.478971,0.977856,-0.595509,pref,pref,1.0


### DT

In [44]:
predict_model(best_models[1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.9076,0.9076,0.9068,0.9068,0.9068,0.8151,0.8151


,average_x,var_abs,range_z,max_x,min_z,max_z,median_abs,range_y,std_x,median_x,average_abs,min_abs,median_z,range_x,max_y,max_abs,fatigue status,Label,Score
0,0.302809,0.383175,2.329275,-1.125984,-2.593261,1.549342,-0.115828,1.328049,-1.737958,0.745661,-0.254219,-1.382665,2.201327,-1.816205,1.101744,0.133481,post,post,1.0
1,0.820430,0.231805,-0.433777,-0.847015,0.267348,-0.440247,-0.525523,-0.122850,0.822879,1.286442,-0.228252,0.197210,0.287812,0.451247,0.936357,0.110321,pref,pref,1.0
2,-0.097086,-0.362411,0.349375,-1.198545,0.142291,0.768653,-0.241917,-0.580970,-0.227414,0.630337,-0.682743,-0.045984,0.759261,-0.530719,0.873072,-0.442057,post,post,1.0
3,0.882774,-0.248497,-1.332795,-0.906435,1.192027,-0.890347,-0.798565,0.446981,-0.730281,1.055298,-0.682070,0.036868,0.416936,-0.566333,1.002208,-0.490266,pref,pref,1.0
4,0.251515,-2.499033,-2.204331,-1.242056,1.455820,-2.134167,-2.269894,-2.277633,-2.281506,-0.019361,-2.388288,1.492297,-0.741366,-2.609826,-1.567937,-2.647125,post,post,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,1.452866,0.929014,-1.093194,-0.848462,0.140169,-2.135501,-0.773128,1.211372,-0.201790,1.416315,-0.071178,0.108084,-2.251777,0.086417,1.131024,0.615118,pref,pref,1.0
234,0.647454,1.213704,1.609738,0.718750,-1.915458,0.893496,0.739140,2.149094,-0.560048,0.017677,0.847435,-0.688200,1.386721,1.109594,-0.656734,-0.236694,post,post,1.0
235,0.968396,0.153740,-1.059784,-0.849844,0.788706,-0.931693,-0.951620,-0.020578,0.128617,1.212644,-0.539768,0.358486,0.380109,0.101774,0.960143,-0.136314,pref,pref,1.0
236,0.819841,0.008660,-1.383519,-0.845820,1.298063,-0.843920,-0.694902,0.238703,-0.214410,1.102260,-0.349707,0.013334,0.401627,-0.478971,0.977856,-0.595509,pref,pref,1.0


### QDA

In [45]:
predict_model(best_models[2])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.8025,0.9117,0.8898,0.7554,0.8171,0.6056,0.6152


,average_x,var_abs,range_z,max_x,min_z,max_z,median_abs,range_y,std_x,median_x,average_abs,min_abs,median_z,range_x,max_y,max_abs,fatigue status,Label,Score
0,0.302809,0.383175,2.329275,-1.125984,-2.593261,1.549342,-0.115828,1.328049,-1.737958,0.745661,-0.254219,-1.382665,2.201327,-1.816205,1.101744,0.133481,post,post,1.0000
1,0.820430,0.231805,-0.433777,-0.847015,0.267348,-0.440247,-0.525523,-0.122850,0.822879,1.286442,-0.228252,0.197210,0.287812,0.451247,0.936357,0.110321,pref,pref,0.9700
2,-0.097086,-0.362411,0.349375,-1.198545,0.142291,0.768653,-0.241917,-0.580970,-0.227414,0.630337,-0.682743,-0.045984,0.759261,-0.530719,0.873072,-0.442057,post,post,0.6155
3,0.882774,-0.248497,-1.332795,-0.906435,1.192027,-0.890347,-0.798565,0.446981,-0.730281,1.055298,-0.682070,0.036868,0.416936,-0.566333,1.002208,-0.490266,pref,pref,0.9912
4,0.251515,-2.499033,-2.204331,-1.242056,1.455820,-2.134167,-2.269894,-2.277633,-2.281506,-0.019361,-2.388288,1.492297,-0.741366,-2.609826,-1.567937,-2.647125,post,pref,0.9991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,1.452866,0.929014,-1.093194,-0.848462,0.140169,-2.135501,-0.773128,1.211372,-0.201790,1.416315,-0.071178,0.108084,-2.251777,0.086417,1.131024,0.615118,pref,pref,0.8874
234,0.647454,1.213704,1.609738,0.718750,-1.915458,0.893496,0.739140,2.149094,-0.560048,0.017677,0.847435,-0.688200,1.386721,1.109594,-0.656734,-0.236694,post,post,1.0000
235,0.968396,0.153740,-1.059784,-0.849844,0.788706,-0.931693,-0.951620,-0.020578,0.128617,1.212644,-0.539768,0.358486,0.380109,0.101774,0.960143,-0.136314,pref,pref,0.9924
236,0.819841,0.008660,-1.383519,-0.845820,1.298063,-0.843920,-0.694902,0.238703,-0.214410,1.102260,-0.349707,0.013334,0.401627,-0.478971,0.977856,-0.595509,pref,pref,0.9984
